In [38]:
import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, scale, MinMaxScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, f1_score
from sklearn.decomposition import PCA

from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import time

pd.options.display.width = 1200
pd.options.display.max_colwidth = 100
pd.options.display.max_columns = 100
pd.options.display.max_rows = 67

In [101]:
data = pd.read_csv("CW_Holdout.csv",index_col=[0,1])
data

,,PP1,PP2,PP3,KF1,KF2,KF3,KF4,KF5,KF6,KF7,KF8,KF9,KF10,Z1,Z2,Z3,Z4,Z5,F1,F2,F3,F4,F5,F6,T1,T2,T3,T4,T5,VHSE1,VHSE2,VHSE3,VHSE4,VHSE5,VHSE6,VHSE7,VHSE8,ProtFP1,ProtFP2,ProtFP3,ProtFP4,ProtFP5,ProtFP6,ProtFP7,ProtFP8,ST1,ST2,ST3,ST4,ST5,ST6,ST7,ST8,BLOSUM1,BLOSUM2,BLOSUM3,BLOSUM4,BLOSUM5,BLOSUM6,BLOSUM7,BLOSUM8,BLOSUM9,BLOSUM10,MSWHIM1,MSWHIM2,MSWHIM3
ID,Epitope,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
EEEIAQVATVSSNGD,E-43592,0.060000,-0.606000,-0.144667,-0.339333,-0.614667,-0.082000,0.362000,-0.359333,-0.106667,-0.416000,0.100667,-0.214667,-0.331333,1.140000,-1.080000,-0.056667,-1.463333,0.376667,NaN,0.137800,-0.514267,0.527067,-0.068133,-0.351533,-5.802000,0.002667,0.558667,0.243333,1.082000,-0.362000,-0.398000,-0.557333,-0.342667,-0.671333,-0.320667,0.220667,-0.164667,-2.074667,-1.451333,-1.579333,-0.122667,-0.138667,0.287333,0.249333,-0.079333,-1.072667,-0.466600,-0.529800,-0.018000,0.188133,-0.572933,0.070000,0.312600,0.516667,-0.378000,-0.054667,0.061333,-0.177333,0.276667,-0.198667,-0.087333,0.382667,0.218667,-0.372667,0.242000,-0.555333
KNEEEIAQVATVSSN,E-43592,0.146667,-0.511333,-0.042667,-0.422000,-0.419333,0.015333,0.486000,-0.190000,-0.192667,-0.332000,-0.011333,-0.113333,-0.484000,1.094000,-0.704000,-0.306000,-1.222000,0.480000,NaN,0.290867,-0.374467,0.452533,-0.023733,-0.120867,-5.265333,0.233333,0.438667,0.232667,0.843333,-0.416000,-0.294000,-0.332667,-0.394667,-0.384667,-0.228000,0.242000,-0.157333,-1.912000,-0.545333,-1.599333,0.463333,-0.150000,0.231333,0.302000,-0.206000,-0.982067,-0.449267,-0.489067,-0.058267,0.197333,-0.280200,0.100933,0.175267,0.533333,-0.378000,-0.248000,0.033333,-0.012000,0.258667,-0.159333,-0.210000,0.324000,0.155333,-0.384000,0.346000,-0.445333
KNSSPVKNEEEIAQV,E-43592,0.170000,-0.438000,-0.026000,-0.220667,-0.228667,-0.092667,0.525333,0.040000,NaN,-0.238000,-0.088000,NaN,NaN,1.070000,-0.326667,-0.314667,-0.969333,0.574000,NaN,0.168400,-0.285133,0.351000,0.209467,-0.116067,-4.773333,0.303333,0.174667,0.146000,0.596000,-0.466667,-0.150667,-0.192667,-0.206000,-0.273333,-0.211333,0.366667,0.094667,-2.359333,0.081333,-1.134000,0.548667,-0.491333,0.098667,0.334000,-0.342667,-0.911400,-0.345467,-0.371200,-0.078600,0.112200,-0.071867,0.128267,0.118933,0.650000,-0.316000,-0.323333,-0.080667,-0.057333,0.092667,-0.112667,-0.090667,0.207333,0.052667,-0.359333,0.330000,-0.344667
PVKNEEEIAQVATVS,E-43592,-0.056000,-0.474000,-0.096000,-0.464000,-0.412000,0.072000,0.327333,-0.096000,NaN,-0.251333,-0.166000,NaN,NaN,0.448000,-0.898667,-0.432000,-1.062667,0.460000,NaN,0.310400,-0.309667,0.500067,0.251067,-0.080667,-5.193333,-0.066000,0.299333,0.308000,0.688667,-0.240000,-0.309333,-0.281333,-0.537333,-0.312667,-0.485333,0.207333,0.124000,-1.200667,-0.776667,-1.486667,0.557333,-0.508667,0.112000,0.322000,-0.165333,-0.978800,-0.511400,-0.410667,-0.024933,0.008000,-0.259933,0.099267,0.087333,0.360000,-0.490000,-0.289333,-0.109333,-0.156000,0.043333,-0.100667,-0.140667,0.351333,0.194000,-0.435333,0.393333,-0.413333
SSPVKNEEEIAQVAT,E-43592,0.038000,-0.500000,-0.048667,-0.360667,-0.436667,-0.053333,0.382000,-0.143333,-0.247333,-0.306000,-0.246667,-0.137333,-0.387333,0.780000,-0.794000,-0.252667,-1.098667,0.506000,NaN,0.217600,-0.383533,0.470467,0.268667,-0.154400,-5.298000,-0.046667,0.326000,0.223333,0.762000,-0.335333,-0.305333,-0.341333,-0.437333,-0.348667,-0.374000,0.180667,0.133333,-1.841333,-0.753333,-1.378667,0.539333,-0.446667,0.038667,0.210000,-0.096667,-0.992800,-0.472600,-0.450133,-0.048400,0.082000,-0.350467,0.111667,0.126733,0.491333,-0.438000,-0.219333,-0.092000,-0.102667,0.068667,-0.100667,-0.195333,0.376000,0.144667,-0.422000,0.381333,-0.441333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DVSLRFGDRKLFEDV,E-9993,0.011333,-0.234000,-0.124667,-0.088000,0.

In [102]:
print("Shape:",data.shape)
print("Total Missing:")
for x, y in enumerate(data.isnull().any()):
    if y == True:
        name = data.columns[x]
        print("   "+name+":", data[name].isnull().sum())

Shape: (15000, 66)
Total Missing:
   KF6: 7667
   KF9: 7667
   KF10: 7667
   F1: 13786
   T2: 504
   ST7: 66


<h3>Predict T2</h3>

In [103]:
from sklearn.preprocessing import scale
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
cols = ["ProtFP2", "MSWHIM3", "ST6", "BLOSUM3", "PP1", "T2"]

new = data[cols]

test = new[new["T2"].isnull()]
train = new.dropna()

X = train.drop("T2", axis=1)
y = train["T2"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

missing = test.drop("T2", axis=1)

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

r2_score(y_test, y_pred)

0.8982535267530695

In [104]:
y_pred = lr.predict(missing)
data.loc[data.T2.isnull(), 'T2'] = y_pred

C:\Users\olive\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2877: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(


<h3>Replace ST7 with mean</h3>

In [105]:
data['ST7'].fillna((data['ST7'].mean()), inplace=True)

<h3>Feature selection</h3>

In [106]:
features = pd.read_pickle("Features")
cols = features.columns.drop("Class")
data = data[cols]
data

,,BLOSUM4,KF3,BLOSUM7,VHSE4,ProtFP7,BLOSUM1,Z3,VHSE8,ST7,T4,F4,T2,BLOSUM3,BLOSUM10,KF1
ID,Epitope,,,,,,,,,,,,,,,
EEEIAQVATVSSNGD,E-43592,0.061333,-0.082000,-0.198667,-0.342667,0.249333,0.516667,-0.056667,-0.164667,0.070000,0.243333,0.527067,0.002667,-0.054667,0.218667,-0.339333
KNEEEIAQVATVSSN,E-43592,0.033333,0.015333,-0.159333,-0.394667,0.302000,0.533333,-0.306000,-0.157333,0.100933,0.232667,0.452533,0.233333,-0.248000,0.155333,-0.422000
KNSSPVKNEEEIAQV,E-43592,-0.080667,-0.092667,-0.112667,-0.206000,0.334000,0.650000,-0.314667,0.094667,0.128267,0.146000,0.351000,0.303333,-0.323333,0.052667,-0.220667
PVKNEEEIAQVATVS,E-43592,-0.109333,0.072000,-0.100667,-0.537333,0.322000,0.360000,-0.432000,0.124000,0.099267,0.308000,0.500067,-0.066000,-0.289333,0.194000,-0.464000
SSPVKNEEEIAQVAT,E-43592,-0.092000,-0.053333,-0.100667,-0.437333,0.210000,0.491333,-0.252667,0.133333,0.111667,0.223333,0.470467,-0.046667,-0.219333,0.144667,-0.360667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DVSLRFGDRKLFEDV,E-9993,0.031333,-0.068667,-0.194667,-0.013333,0.147333,0.177333,-0.416000,0.037333,0.106467,0.066000,0.465600,0.608667,-0.248000,-0.248000,-0.088000
GDRKLFEDVNIKFTE,E-9993,0.036667,-0.116000,-0.223333,0.043333,0.248667,0.310667,-0.478667,-0.045333,0.062867,0.048000,0.346600,0.648000,-0.279333,-0.253333,-0.151333
RFGDRKLFEDVNIKF,E-9993,0.062000,-0.022000,-0.093333,0.164000,0.171333,0.158667,-0.559333,-0.030667,0.120933,-0.044667,0.286067,0.764667,-0.324667,-0.398000,-0.071333


<h3>Scale Data</h3>

In [107]:
scaler = MinMaxScaler()
scaler.fit(data[cols])
data[cols] = scaler.transform(data[cols])

<ipython-input-107-9c18250707b0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[cols] = scaler.transform(data[cols])
C:\Users\olive\anaconda3\lib\site-packages\pandas\core\frame.py:3513: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.loc._setitem_with_indexer((slice(None), indexer), value)
C:\Users\olive\anaconda3\lib\site-packages\pandas\core\frame.py:3484: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

In [99]:
trainer = pd.read_pickle("data_combined")
trainer

,BLOSUM4,KF3,BLOSUM7,VHSE4,ProtFP7,BLOSUM1,Z3,VHSE8,ST7,T4,F4,T2,BLOSUM3,BLOSUM10,KF1,Class
0,0.319087,0.319924,0.423611,0.649659,0.538047,0.649828,0.244074,0.238011,0.457679,0.524941,0.641526,0.858511,0.224756,0.242321,0.411765,Negative
1,0.280445,0.370778,0.503704,0.626224,0.533291,0.571542,0.184021,0.208228,0.458730,0.597038,0.707107,0.850613,0.175896,0.260068,0.365729,Negative
2,0.351874,0.455408,0.538889,0.656482,0.649017,0.588469,0.169447,0.240788,0.427671,0.507404,0.609287,0.888590,0.113192,0.036177,0.460642,Negative
3,0.227166,0.375712,0.475926,0.678434,0.440393,0.649564,0.219315,0.212014,0.516383,0.532736,0.543831,0.886574,0.172638,0.333106,0.371696,Negative
4,0.311475,0.398482,0.488889,0.599229,0.594800,0.520233,0.166813,0.242554,0.398844,0.533905,0.678086,0.873467,0.084691,0.122867,0.367150,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25947,0.407860,0.369663,0.504138,0.578262,0.560108,0.726688,0.413698,0.304685,0.469121,0.506963,0.663905,0.626695,0.472687,0.582893,0.476360,Positive
25948,0.375030,0.471556,0.642110,0.300228,0.505407,0.347140,0.319031,0.250660,0.371283,0.777248,0.839330,0.491947,0.323841,0.598128,0.238866,Positive
25949,0.502342,0.673245,0.595833,0.606942,0.596386,0.521026,0.328885,0.221605,0.396677,0.357755,0.467525,0.713830,0.297231,0.445734,0.506678,Positive
25950,0.893730,0.676244,0.506734,0.530206,0.862578,0.788115,0.549383,0.169522,0.307999,0.461809,0.512404,0.583592,0.543132,0.466521,0.784701,Positive


<h3>Predict data</h3>

In [111]:
X_train = trainer.drop(columns='Class')
y_train = trainer["Class"]

model = KNeighborsClassifier(weights="distance", n_neighbors=6)
model.fit(X_train,y_train)
pred_y = model.predict(data)

In [112]:
#Shave_Oliver_CS4850_predictions.csv
predicted = data.copy()
predicted.insert(15, "Class", pred_y)

In [113]:
predicted

,,BLOSUM4,KF3,BLOSUM7,VHSE4,ProtFP7,BLOSUM1,Z3,VHSE8,ST7,T4,F4,T2,BLOSUM3,BLOSUM10,KF1,Class
ID,Epitope,,,,,,,,,,,,,,,,
EEEIAQVATVSSNGD,E-43592,0.431163,0.427960,0.308052,0.302660,0.664975,0.659243,0.460228,0.239163,0.373661,0.569389,0.644696,0.562763,0.438780,0.636463,0.310979,Positive
KNEEEIAQVATVSSN,E-43592,0.411628,0.480029,0.335674,0.281915,0.686081,0.665951,0.394557,0.241903,0.403425,0.563045,0.610878,0.620904,0.322360,0.584294,0.274184,Negative
KNSSPVKNEEEIAQV,E-43592,0.332093,0.422254,0.368446,0.357181,0.698905,0.712906,0.392274,0.336074,0.429726,0.511499,0.564808,0.638548,0.276997,0.499725,0.363798,Negative
PVKNEEEIAQVATVS,E-43592,0.312093,0.510342,0.376873,0.225000,0.694096,0.596190,0.361370,0.347035,0.401822,0.607851,0.632445,0.545455,0.297471,0.616145,0.255490,Positive
SSPVKNEEEIAQVAT,E-43592,0.324186,0.443295,0.376873,0.264894,0.649212,0.649047,0.408604,0.350523,0.413753,0.557494,0.619014,0.550328,0.339623,0.575508,0.301484,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DVSLRFGDRKLFEDV,E-9993,0.410233,0.435093,0.310861,0.434043,0.624098,0.522672,0.365584,0.314649,0.408750,0.463918,0.616806,0.715510,0.322360,0.252059,0.422849,Negative
GDRKLFEDVNIKFTE,E-9993,0.413953,0.409772,0.290730,0.456649,0.664707,0.576335,0.349078,0.283757,0.366797,0.453212,0.562812,0.725424,0.303493,0.247666,0.394659,Negative
RFGDRKLFEDVNIKF,E-9993,0.431628,0.460057,0.382022,0.504787,0.633716,0.515160,0.327831,0.289238,0.422670,0.398097,0.535346,0.754831,0.276194,0.128501,0.430267,Negative


In [114]:
predicted.to_csv("Predictions.csv")